In [1]:

import pandas as pd
import numpy as np
import os
import openpyxl as xl

import re

import warnings
warnings.filterwarnings('ignore')

from Utilities import FileIO
from Utilities import DataframeCleaning


# Load File

In [2]:
df_raw = FileIO.importFile("transformFinished_separate.csv")

df_raw.head()


,product_id,product,mail_class,fy,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,volume,weight,weight_pc
0,3,Single Piece Letters,First Class,2008,14353.524033,8443.075618748211,8434.32261874821,8.753,0.4283392480234627,0.2519590766159719,0.1763801714074908,1.7000349968591284,3.350971e+07,1000328.975,0.477631
1,4,Single Piece Cards,First Class,2008,500.489597,446.5087719559969,446.02677195599694,0.482,0.27114174086745557,0.2418974669332412,0.02924427393421436,1.12089532935161,1.845860e+06,11830.080,0.102544
2,5,Total Single Piece Letters and Cards,First Class,2008,14854.01363,8889.584390704207,8880.349390704207,9.235,0.42013220917755617,0.25143377552810814,0.16869843364944803,1.6709457919689432,3.535557e+07,1012159.055,0.458048
3,8,Presort Letters,First Class,2008,16327.804358,5441.6680782620415,5429.030078262042,12.638,0.3374916702247292,0.11247793079057965,0.22501373943414954,3.000514570748088,4.837987e+07,2174874.013,0.719266
4,9,Presort Cards,First Class,2008,732.236796,282.27046183504393,281.3414618350439,0.929,0.20591601550888014,0.07937870524185411,0.12653731026702603,2.594096425250163,3.555997e+06,28969.254,0.130345


# Convert Numeric Columns

In [3]:
df = df_raw.copy()

def convertNumericColumns(frame):
    nonNumericCols = ['product_id', 'product', 'mail_class', 'fy']
    numericCols = [c for c in frame.columns if c not in nonNumericCols]

    def applyNumConversion(row):
        rowString = str(row)
        deleteChars = [",", "$"]
        for char in deleteChars:
            rowString = rowString.replace(char, "")

        if rowString in ["None", "NaN", "(D-E)", "(D/E)"]:
            return np.nan
        
        if len(rowString)<2:
            return np.nan
        
        rowFloat = float(rowString)

        return rowFloat


    for col in numericCols: 
        frame[col] = frame[col].apply(lambda row: applyNumConversion(row))
    
    return frame


df_converted = convertNumericColumns(df)

# df_converted.info()
# df.info()

# Correct Units

In [4]:
df = df_converted.copy()

millionCols = ['revenue',
       'attributable_cost', 'volume_variable_cost', 'product_specific_cost']

def correctUnits(frame):


    thousandCols = ['volume','weight']

    million = 1000000
    thousand = 1000

    for col in millionCols:
        frame[col] =frame[col]*million
    
    for col in thousandCols:
        frame[col] =frame[col]*thousand


    return frame

df_correctUnits = correctUnits(df)
# df_correctUnits.head()
df.columns

Index(['product_id', 'product', 'mail_class', 'fy', 'revenue',
       'attributable_cost', 'volume_variable_cost', 'product_specific_cost',
       'rev_pc', 'attributable_cost_pc', 'cont_pc', 'cost_coverage', 'volume',
       'weight', 'weight_pc'],
      dtype='object')

# Round Columns

In [5]:
df = df_correctUnits.copy()

identifierColumns = ['product_id', 'product', 'mail_class', 'fy']


def roundColumns(frame):

    valueColumns = [c for c in frame.columns if c not in  identifierColumns]

    for col in valueColumns:
        frame[col] = frame[col].round(6) 

    return frame

df_rounded = roundColumns(df)
# df_rounded.head()

# Reassign Current Totals as International

In [6]:
keyFrame = FileIO.importKey()



keyFilt =  keyFrame[keyFrame['notes'].notnull()]
keyFilt =keyFilt[keyFilt['notes'].str.contains("sum of ids")][['product_id','product','notes']]

def applyNumberMatch(row):
    numbers =  re.findall(r"\d+",row)
    return numbers

keyFilt['children'] = keyFilt['notes'].apply(lambda row: applyNumberMatch(row))
# keyFilt


summaryRowsDictStringList= dict(
    zip(
        list(keyFilt['product_id']), 
        list(keyFilt['children'])
    )
)


summaryRowsDict = {}
for keyDict in summaryRowsDictStringList.keys():
    stringList= summaryRowsDictStringList[keyDict]
    intList = [int(n) for n in stringList]
    summaryRowsDict[keyDict] = intList


summaryRowsDict


{80: [5, 10, 14, 19, 222, 230, 103, 127],
 90: [80, 81, 82, 83, 85, 86],
 92: [90, 91],
 248: [227, 101, 102, 220, 92],
 81: [21, 22, 23, 24, 25, 26, 27, 104, 139, 231, 223],
 83: [41, 42, 43, 44, 45, 226, 233, 106, 253],
 82: [31, 32, 224, 105, 232],
 91: [51, 52, 54, 55, 56, 57, 58, 61, 62, 73, 74, 142, 225, 76]}

In [7]:
df = df_rounded.copy()




summaryReassignDict = {80: 262 ,
81: 263, 
83: 265, 
82: 264,
91: 266}

def reassignSummaryRows(frame):
    def applyReassign(row):
        if row in summaryReassignDict.keys():
            return summaryReassignDict[row]
        return row
    
    frame['product_id'] = frame['product_id'].apply(lambda row: applyReassign(row))

    frame = DataframeCleaning.realignWithKey(frame, keyFrame)

    return frame


df_reassignedSummary = reassignSummaryRows(df)

# df_reassignedSummary[df_reassignedSummary['product_id']==266]
# # df_reassignedSummary[df_reassignedSummary['product_id'].isin(summaryReassignDict.values())]

# Calculate Domestic Totals

In [8]:
df = df_reassignedSummary.copy()

def calculateDomesticTotals(frame, idToRecalculate, childrenList):
    frameOrig = frame.copy()
    frame = frame[frame['product_id']!= idToRecalculate]
    frameIdListFilter = frame['product_id'].isin(childrenList)
    valueColumns = [c for c in frame.columns if c not in  identifierColumns]

    grp = frame[frameIdListFilter].groupby("fy")[valueColumns].sum()
    grp.reset_index(inplace=True)

    grp['product_id'] = idToRecalculate

    for col in frameOrig.columns:
        if col not in grp:
            grp[col] = np.nan
    
    rez = pd.concat([frame, grp])

    return rez

def calcDomesticTotals_aggregate(frame):
    for keyDict in summaryRowsDict.keys():
        summaryRow = keyDict
        childrenList = summaryRowsDict[keyDict]
        frame = calculateDomesticTotals(frame, summaryRow, childrenList)

    rez = DataframeCleaning.realignWithKey(frame, keyFrame)
    return rez


df_domesticSums = calcDomesticTotals_aggregate(df)


df = df_domesticSums.copy()

df[df['product_id'].isin(summaryRowsDict.keys())].head()


,product_id,product,mail_class,fy,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,volume,weight,weight_pc
958,80,Total First Class (Domestic),Summary,2008,3.727660e+10,1.826423e+10,1.824039e+10,2.384100e+07,3.799691,2.905515,0.894177,7.267123,9.127670e+10,4.104553e+09,9.573919
959,80,Total First Class (Domestic),Summary,2009,3.495505e+10,1.724019e+10,1.720813e+10,3.205207e+07,4.256290,3.146420,1.109870,9.797005,8.331373e+10,3.634747e+09,10.317914
960,80,Total First Class (Domestic),Summary,2010,3.313107e+10,1.645466e+10,1.642512e+10,2.954422e+07,4.016086,3.224905,0.791181,7.049557,7.786872e+10,3.635889e+09,9.835242
961,80,Total First Class (Domestic),Summary,2011,3.131395e+10,1.548361e+10,1.545043e+10,3.317751e+07,4.070924,3.099234,0.971692,7.144492,7.321016e+10,3.675243e+09,9.786556
962,80,Total First Class (Domestic),Summary,2012,2.951612e+10,1.421490e+10,1.416946e+10,4.544946e+07,4.689478,3.645034,1.044444,9.905881,6.898454e+10,3.400451e+09,10.542421


In [12]:
isYr  = df['fy']==2023
isMM = df['mail_class'].str.contains("Mark")

tempFrame = df[isYr & isMM]
vals = list(tempFrame[['revenue','attributable_cost']].sum().values)
vals[0]/vals[1]



1.4188262041089261

In [9]:

stop 

NameError: name 'stop' is not defined

# Reorder Columns



In [ ]:
df = df_domesticSums.copy()

def reorderCols(frame):
    leadingCols = ['product_id', 'product', 'mail_class', 'fy', 'volume', 'revenue',
       'attributable_cost']
    
    trailingCols = [c for c in frame.columns if c not in leadingCols]
    finalCols = leadingCols + trailingCols
    return frame[finalCols]

# reorderCols(df)
# df.columns

# Aggregate Funcs

In [ ]:
df_raw=  FileIO.importFile("transformFinished_separate.csv")


transformFuncs = [
convertNumericColumns,
correctUnits,
roundColumns,
reassignSummaryRows,
calcDomesticTotals_aggregate,
reorderCols,
DataframeCleaning.realignWithKey
]

def applyTransformations(frame, funcs):
    for func in funcs:
        frame = func(frame)
    return frame

df_final = applyTransformations(df_raw, transformFuncs)

df_final.head()



,product_id,product,mail_class,fy,volume,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,weight,weight_pc
0,3,Single Piece Letters,First Class,2008,3.350971e+10,1.435352e+10,8.443076e+09,8.434323e+09,8753000.0,0.428339,0.251959,0.176380,1.700035,1.000329e+09,0.477631
1,4,Single Piece Cards,First Class,2008,1.845860e+09,5.004896e+08,4.465088e+08,4.460268e+08,482000.0,0.271142,0.241897,0.029244,1.120895,1.183008e+07,0.102544
2,5,Total Single Piece Letters and Cards,First Class,2008,3.535557e+10,1.485401e+10,8.889584e+09,8.880349e+09,9235000.0,0.420132,0.251434,0.168698,1.670946,1.012159e+09,0.458048
3,8,Presort Letters,First Class,2008,4.837987e+10,1.632780e+10,5.441668e+09,5.429030e+09,12638000.0,0.337492,0.112478,0.225014,3.000515,2.174874e+09,0.719266
4,9,Presort Cards,First Class,2008,3.555997e+09,7.322368e+08,2.822705e+08,2.813415e+08,929000.0,0.205916,0.079379,0.126537,2.594096,2.896925e+07,0.130345


# Upload File


In [ ]:

FileIO.exportFile(df_final, "transformFinished_merged.csv")

# Garbage

In [ ]:
df  = df_final.copy()
# # df= df_raw.copy()
# df[df['product_id']==262]

# df[df['product'].str.contains("Cre")]

# df_raw[df_raw['product'].str.contains("Cre")]
# df_raw[df_raw['product'].str.contains("Ch")]

countSer = df.groupby(["fy", "product_id"])['product'].count()
countSer.loc[countSer>1]

Series([], Name: product, dtype: int64)

In [ ]:
isYearRaw = df_raw['fy']==2010
isIdRaw = df_raw['product_id']==3
df_raw[isYearRaw & isIdRaw]

,product_id,product,mail_class,fy,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,volume,weight,weight_pc
87,3,Single Piece Letters,First Class,2010,1.233956e+10,7.376828e+09,7.354369e+09,22459501.0,0.45453,0.271727,0.182803,1.672745,2.714792e+10,847776797.0,0.499649
